# Алгоритм Томпсона на батчах (без контекста)

1. На первом батче распределяем юзеров 50% на 50%.
2. Вероятность конверсии каждого варианта распределена по Beta распределению с $\alpha=1$,
$\beta=1$.
3. В конце каждого батча пересчитываем вероятности превосходства по точной формуле,
взятой отсюда https://www.johndcook.com//UTMDABTR-005-05.pdf
4. Распределяем трафик в пропорции вероятностей превосходства для каждого варианта
5. Останавливаем эксперимент при достижении определенной вероятности превосходства,
но не раньше определенного дня, чтобы учесть календарные факторы

Потенциальные проблемы:
- слишком рано отдаем трафик победителю
- из-за дисбаланса распределения трафика может быть больше успешных конверсий в этом варианте
(можно попробовать применить нормализацию)


***Реализуем алгоритм***

0. **Инициализация** - *BatchThompson(n_arms)*. Аргумент на вход: число вариантов сплита.
Здесь также инициализируются массивы для параметров Бета-распределений и вероятность превосходства = 0.5.

1. **Метод сплита** - *split_data()*. Исходя из вероятности превосходства вычисляем сплит по вариантам.
Возвращаем данные по конверсии на текущем батче для пересчета Бета-распределений.

2. **Метод изменения параметров распределения** - *.update_beta_params(data)*. Аргументы на вход: numpy массив со значениями конверсии по
каждому варианту. В случае неравномерного распределения по вариантам ставятся пропуски.
 - Проверяем, чтобы число столбцов совпадало с числом вариантов из инициализации.
 - Суммируем нули и единицы и обновляем параметры

3. **Метод пересчета** - *update_prob_super()*. Аргументов нет, так как учитывает измененные параметры
$\alpha$ (накопленное число успешных конверсий) и
$\beta$ (накопленное число неудачных конверсий) для всех вариантов.
 - Считаем по точной формуле
 - Выдаем массив из вероятностей превосходства

4. **Вероятность превосходства** - *prob_super_tuple()*. Аргументов нет, так как берем пересчитанные параметры.
5. **Критерий остановки** (*stopping_criterion*) - условия цикла while. Либо вероятность превосходства выше заданной
величины, либо закончились наблюдения.

In [26]:
from typing import List, Tuple
import os
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
from numpy import ndarray
from tqdm.notebook import tqdm
from src.ab import get_size_zratio
from src.mab import calc_prob_between

# Графики
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore")
from src.mab import plot_mab_results
from src.mab import BatchThompson
from matplotlib.backends.backend_pdf import PdfPages
from joblib import Parallel, delayed

# Эксперименты на разных теоретических конверсиях и размерах батча (summation vs normalization)

In [27]:
p1_control = np.round(np.linspace(0.01, 0.5, 3), 3)
mde_test_effect = np.round(np.linspace(0.01, 0.15, 3), 3)
tuple_batch_size_share = np.round(np.linspace(0.001, 0.1, 1), 3)

result_experiments_df = pd.DataFrame(index=pd.MultiIndex.from_product(
                                     [p1_control, mde_test_effect, tuple_batch_size_share],
                                     names=["p1", "mde", "batch_size_share"]),
                                     columns=['probability_superiority_steps',
                                              'cumulative_observations_step_list',
                                              'n_obs_per_every_arm'])
plot_file = PdfPages("/home/igor/Appbooster/proba.ai/AB/Plot/Thompson/Experiment3/Thompson.pdf")
def thompson_results(index):
    p_list = [index[0], index[0] * (1 + index[1])]
    batch_size_share = index[2]
    bts = BatchThompson(p_list_mu=p_list, batch_size_share_mu=batch_size_share)
    probability_superiority_steps, observations_step_list = bts.start_experiment()
    cumulative_observations_step_list = np.cumsum(observations_step_list, axis=0)
    # plot_file.savefig(plot_mab_results(p_list, batch_size_share,
    #                                    probability_superiority_steps, cumulative_observations_step_list));
    # plt.close();
    return (p_list, batch_size_share,
            probability_superiority_steps,
            cumulative_observations_step_list
            )
plot_file = PdfPages("/home/igor/Appbooster/proba.ai/AB/Plot/Thompson/Experiment3/Thompson.pdf")
results_all =  Parallel(n_jobs=-1)(
                             delayed(thompson_results)(index)
                             for index, row in result_experiments_df.iterrows()
                             )
# for index, row in  tqdm(result_experiments_df.iterrows()):
#
#     p_list = [index[0], index[0] * (1 + index[1])]
#     batch_size_share = index[2]
#     bts = BatchThompson(p_list_mu=p_list, batch_size_share_mu=batch_size_share)
#     probability_superiority_steps, observations_step_list = bts.start_experiment()
#     cumulative_observations_step_list = np.cumsum(observations_step_list, axis=0)
#     result_experiments_df.loc[index, "n_obs_per_every_arm"] = bts.n_obs_every_arm
#     result_experiments_df.loc[index, "probability_superiority_steps"] = probability_superiority_steps
#     result_experiments_df.loc[index, "cumulative_observations_step_list"] = cumulative_observations_step_list
#     plot_file.savefig(plot_mab_results(p_list, batch_size_share,
#                                        probability_superiority_steps, cumulative_observations_step_list));
#     plt.close();
# plot_file.close()

In [35]:
i = 0
for index, row in  tqdm(result_experiments_df.iterrows()):
    result_experiments_df.loc[index, "probability_superiority_steps"] = results_all[i][2]
    result_experiments_df.loc[index, "cumulative_observations_step_list"] = results_all[i][3]
    i += 1

0it [00:00, ?it/s]

Попробуем реализовать метод, описанный в статье
https://www.researchgate.net/publication/352117401_Parallelizing_Thompson_Sampling

Авторы предлагают следующий алгоритм. Пусть $k_a$ - число раз выбора руки $a$,
$l_a = 1$ - число раз выбора руки подряд.

Для каждого батча $t = 1, 2, .. T$:

- смотрим на вероятности бета распределений
- выбираем руку с наибольшей вероятностью
- присваиваем $k_a = k_a + 1$
- ЕСЛИ $k_a < 2^{l_a}$, то кидаем ВЕСЬ трафик в эту руку
- ИНАЧЕ: присваиваем $l_a = l_a + 1$ и распределяем трафик в ОБЕ руки ПОЛНОСТЬЮ (без долей)
- обновляем параметры и по новой

Утверждается, что он довольно хорошо работает и для динамических батчей - когда размер заранее нам неизвестен

In [64]:
def all_equal(lst):
    for arr in lst[1:]:
        if not np.array_equal(lst[0], arr, equal_nan=True):
            return False
    return True


class BatchThompson1(BatchThompson):

    def split_data_random(self, best_arms: np.array):
        data_split = np.empty((self.batch_size, self.n_arms))
        data_split[:] = np.nan
        for i in range(self.n_arms):
            if i in best_arms:
                data_split[:, i] = np.random.binomial(n=1, p=self.p_list[i], size=self.batch_size)
            else:
                data_split[:, i] = np.nan
        return data_split


    def start_experiment(self):
        cumulative_observations = np.repeat(0, self.n_arms)
        probability_superiority_step_list: List[ndarray] = []  # how share of traffic changes across experiment
        observations_step_list = []
        k_list = [0] * self.n_arms
        l_list = [0] * self.n_arms

        for i in tqdm(range(0, np.uint16(1 / (self.batch_size_share / self.n_arms)))):

            # Determine argmax arm
            if all_equal(self.probability_superiority_tuple):
                best_arm = np.random.choice(len(self.probability_superiority_tuple[:-1]),
                                            size=len(self.probability_superiority_tuple[:-1]))[0]
            else:
                best_arm = np.argmax(self.probability_superiority_tuple)
            k_list[best_arm] += 1
            if k_list[best_arm] == 2 ** l_list[best_arm]:
                l_list[best_arm] += 1
                batch_data = self.split_data_random(best_arms=np.arange(self.n_arms))  # based on generate batch online
            elif k_list[best_arm] < 2 ** l_list[best_arm]:
                batch_data = self.split_data_random(best_arms=np.array(best_arm))

            batch_non_zero_observations_step = batch_data.shape[0] - np.isnan(batch_data).sum(axis=0)
            cumulative_observations += batch_non_zero_observations_step
            observations_step_list.append(batch_non_zero_observations_step)
            # Updating all
            self.update_beta_params(batch_data, method="summation")  # update beta distributions parameters
            self.update_prob_super(method_calc="integrating") # update probability superiority

            # Append for resulting
            probability_superiority_step_list.append(self.probability_superiority_tuple)

            stopping_criterion = (np.max(self.probability_superiority_tuple) >= 0.99) | \
                                 (np.max(cumulative_observations) >  self.n_obs_every_arm)
            if stopping_criterion:
                break

        return np.round(probability_superiority_step_list, 3), observations_step_list,\
               k_list, l_list
bt1 = BatchThompson1(p_list=[0.4, 0.42], batch_size_share=0.0)
probability_superiority_experiment, observations_step_list, k_list, l_list = bt1.start_experiment()
print(probability_superiority_experiment)
print(observations_step_list)
print(k_list)
print(l_list)

  0%|          | 0/20 [00:00<?, ?it/s]

[[0.083 0.917]
 [0.013 0.987]
 [0.001 0.999]]
[array([1897, 1897]), array([1897, 1897]), array([1897, 1897])]
[1, 2]
[1, 2]


Проблема такого подхода - отдаем не в пропорциях, а полностью в какой-то батч.
Попробуем объединить алгоритмы: будем разделять трафик согласно пропорциям,
но с учетом накопленных побед для каждой руки. Соответственно, если число побед руки будет равно какому-то
числу, которое зависит от шага, то меняем сплит 50 на 50.
Эксперимент проводим на стохастической конверсии:
- конверсия распределена случайно (экспоненциальное распределение) с неким мат ожиданием
- размер батча будет случайной величиной из нормального распределения с мат ожиданием и дисперсией

In [ ]:
class BatchThompsonMixed:
    def __init__(self, mu_p_list: List, mu_batch_size: int):



    def split_data_random(self, best_arms: np.array, p_):
        data_split = np.empty((self.batch_size, self.n_arms))
        data_split[:] = np.nan
        for i in range(self.n_arms):
            if i in best_arms:
                data_split[:, i] = np.random.binomial(n=1, p=self.p_list[i], size=self.batch_size)
            else:
                data_split[:, i] = np.nan
        return data_split

In [26]:
np.random.exponential(scale=0.02, size=1).mean()

0.03515063035170591